In [10]:
# Define the gpu  on the gpu machine
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

import pandas as pd
import json
from tqdm import tqdm
from transformers import AutoTokenizer
from datasets import load_dataset
import numpy as np

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [18]:
# Import the train dataset
train = load_dataset("TajaKuzman/X-GENRE-multilingual-text-genre-dataset", "train")

# To open as Pandas DataFrame:
train_df = pd.DataFrame(train["train"])

display(train_df.head(2))

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Tokenize the train dataset
tokens_train = []
integers_train = []
token_list_train = []

for text in tqdm(train_df.text.to_list()):
	encoded_text = tokenizer(text)
	# Take all tokens_train, except the beginning (<s>) and end (</s>) token
	current_tokens_train = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)[1:-1]
	tokens_train.append(current_tokens_train)
	token_list_train.extend(current_tokens_train)
	integers_train.append(encoded_text.input_ids[1:-1])

train_df["tokens_train"] = tokens_train
train_df["token_ids"] = integers_train

#print(token_list_train[:10])
#print(len(token_list_train))

# Create a list of tokens, where we take only the first 512 tokens
train_tokens_shortened = []

for i in train_df["tokens_train"].to_list():
	train_tokens_shortened.extend(i[:512])

print(len(train_tokens_shortened))

train_df.head(3)

# Save the tokenized version
train_df.to_json("datasets/tokenized_datasets/X-GENRE-train-tokenized.json")

,text,labels,dataset,language
0,"Seeking All Things Brilliant ""I want people to...",Other,CORE,English
1,Meet Orchid du Bois I first met Hayley Mowday ...,Other,CORE,English


100%|██████████| 1772/1772 [00:09<00:00, 189.74it/s]


699465


In [7]:
# Import the final dataset with test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json") as main_file:
	main_dict = json.load(main_file)

main_dict.keys()

dict_keys(['mt', 'el', 'tr', 'sq', 'is', 'uk', 'ca', 'mk', 'hr', 'sl'])

In [20]:
# Define an array of token overlap
token_overlap_results = {}

# Loop through the datasets and calculate token overlap
for lang in list(main_dict.keys()):
	df = pd.DataFrame(main_dict[lang]["dataset"])

	tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

	tokens = []
	integers = []
	token_list = []

	print("Tokenizing text.")

	for text in tqdm(df.text.to_list()):
		encoded_text = tokenizer(text)
		# Take all tokens, except the beginning (<s>) and end (</s>) token
		current_tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)[1:-1]
		# Shorten the list to 512, as tokens after that were not observed by the classifier
		current_tokens = current_tokens[:512]
		tokens.append(current_tokens)
		token_list.extend(current_tokens)
		integers.append(encoded_text.input_ids[1:-1][:512])

	df["tokens"] = tokens
	df["token_ids"] = integers

	print(token_list[:10])
	print("All tokens:")
	print(len(token_list))

	# See how many tokens overlap
	overlap_counter = 0

	print("Calculating overlap.")

	# Save tokens that overlap for further inspection
	overlap_token_list = []

	#overlap_counter = sum(1 for element in token_list if element in train_tokens_shortened)
	for token in tqdm(token_list):
		if token in train_tokens_shortened:
			overlap_counter += 1
			overlap_token_list.append(token)

	# Out of all tokens, how many overlap?
	overlap_per = overlap_counter/len(token_list)

	print(f"Number of tokens that overlap: {overlap_counter}")
	print(f"Percentage of overlap: {overlap_per}")

	# Update the dataset in the dictionary
	main_dict[lang]["dataset"] = df.to_dict()

	# Add the list of all tokens to the dictionary
	main_dict[lang]["token_overlap"] = {"overlap_percentage":overlap_per, "token_list": token_list, "overlap_token_list":overlap_token_list}

	# Add to the results
	token_overlap_results[lang] = {"percentage": overlap_per, "overlap_list": overlap_token_list}

	# Convert tokens back to words
	#print(tokenizer.convert_tokens_to_string(tokens))

# Inspect the results
overlap_df = pd.DataFrame(token_overlap_results)
	

Tokenizing text.


100%|██████████| 80/80 [00:00<00:00, 315.86it/s]


['▁Angel', 'o', '▁Che', 't', 'cuti', ',', '▁se', '▁j', 'kun', '▁qe']
All tokens:
39040
Calculating overlap.


100%|██████████| 39040/39040 [01:21<00:00, 478.57it/s] 


Number of tokens that overlap: 31899
Percentage of overlap: 0.8170850409836066
Tokenizing text.


...

Number of tokens that overlap: 21808
Percentage of overlap: 0.8215173660815188
Tokenizing text.


100%|██████████| 80/80 [00:00<00:00, 656.03it/s]


['▁Kita', 'jsko', '▁mesto', '▁duhov', '▁V', '▁Notranj', 'i', '▁Mongol', 'iji', '▁raste']
All tokens:
26292
Calculating overlap.


100%|██████████| 26292/26292 [00:45<00:00, 573.42it/s]

Number of tokens that overlap: 25616
Percentage of overlap: 0.9742887570363609


In [26]:
overlap_df = pd.DataFrame(token_overlap_results).transpose()
overlap_df

,percentage,overlap_list
mt,0.817085,"[▁Angel, o, ▁Che, t, ,, ▁se, ▁j, kun, d, u, ▁p..."
el,0.161428,"[asi, asi, ,, ,, ▁Re, ception, ., ▁driver, ▁es..."
tr,0.521502,"[▁A, L, ▁Der, s, i, ▁ve, ▁Beli, r, leme, ▁S, h..."
sq,0.605775,"[▁Blog, ▁“, U, ▁kam, ▁me, jet, .”, ▁Jer, ▁31, ..."
is,0.517575,"[▁[, is, ], ▁er, fi, ▁reg, ▁sett, ar, lag, sin..."
uk,0.156675,"[., ,, ,, ?, ▁-, ., ?, ▁-, ', ., ,, ., ▁, ▁(, ..."
ca,0.744881,"[▁P, à, gine, s, ▁En, nada, ▁Porto, ▁uns, ▁die..."
mk,0.145989,"[,, ▁T, CL, ,, ▁T, CL, :, ▁Alca, tel, ▁Mobile,..."
hr,0.821517,"[▁O, ▁proizvod, u, ▁Color, ▁Trans, ,, ▁za, ▁pa..."
sl,0.974289,"[▁Kita, jsko, ▁mesto, ▁duhov, ▁V, ▁Notranj, i,..."


In [36]:
for token in overlap_df["overlap_list"].to_list()[0][:2]:
	if len(token) > 1:
		non_short_counter += 1

▁Angel


In [37]:
# Inspect how many tokens are one character long

counters = []
pers = []

for token_list_el in overlap_df["overlap_list"].to_list():
	non_short_counter = 0
	for token in token_list_el:
		if len(token) > 1:
			non_short_counter += 1
	
	counters.append(non_short_counter)
	pers.append(non_short_counter/len(token_list_el))

# Add to df
overlap_df["non_short"] = counters

# Calculate the percentage of overlap tokens that have more than 1 character
overlap_df["non_short_per"] = pers

overlap_df


,percentage,overlap_list,non_short,non_short_per
mt,0.817085,"[▁Angel, o, ▁Che, t, ,, ▁se, ▁j, kun, d, u, ▁p...",23083,0.723628
el,0.161428,"[asi, asi, ,, ,, ▁Re, ception, ., ▁driver, ▁es...",1883,0.373389
tr,0.521502,"[▁A, L, ▁Der, s, i, ▁ve, ▁Beli, r, leme, ▁S, h...",11141,0.722269
sq,0.605775,"[▁Blog, ▁“, U, ▁kam, ▁me, jet, .”, ▁Jer, ▁31, ...",12459,0.768315
is,0.517575,"[▁[, is, ], ▁er, fi, ▁reg, ▁sett, ar, lag, sin...",10915,0.711399
uk,0.156675,"[., ,, ,, ?, ▁-, ., ?, ▁-, ', ., ,, ., ▁, ▁(, ...",1258,0.253476
ca,0.744881,"[▁P, à, gine, s, ▁En, nada, ▁Porto, ▁uns, ▁die...",15657,0.763123
mk,0.145989,"[,, ▁T, CL, ,, ▁T, CL, :, ▁Alca, tel, ▁Mobile,...",1270,0.314746
hr,0.821517,"[▁O, ▁proizvod, u, ▁Color, ▁Trans, ,, ▁za, ▁pa...",17678,0.810620
sl,0.974289,"[▁Kita, jsko, ▁mesto, ▁duhov, ▁V, ▁Notranj, i,...",21567,0.841935


In [ ]:
# Calculate token distribution - vector of values per tokens (how many times does each token occur) - create a dictionary with all tokens from train set and specific test set, iterate through the tokens and count how many times each ocurrs. Calculate cosine similarity.

# Do this on label level as well to see whether this explains good performance on some of the labels for Maltese.

In [ ]:
# Transliterate Cyrillic and Greek and see whether the overlap effect changes. If you find a proper library, you can also normalize specific letters (š -> s ...)

In [49]:
# Save the extended json dict
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "w") as file:
	json.dump(main_dict, file)